In [ ]:
import numpy as np

import getpass
user = getpass.getuser()

In [ ]:
from PIL import Image

In [ ]:
from omni.isaac.kit import SimulationApp

# "/home/yizhou/Research/OpenAnyDrawer/scene0.usd" # 
usd_path = f"omniverse://localhost/Users/{user}/scene0.usd"


simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 


# World

In [ ]:
from omni.isaac.core import World
world = World()

In [ ]:
from open_env import OpenEnv

env = OpenEnv()

In [ ]:
from franka.control import FrankaControl

controller = FrankaControl("/World/Game*/Franka", "/World/Game*/Franka/panda_hand")

In [ ]:
env.setup_viewport()

In [ ]:
env.get_image()

# Add Object/Robot

In [ ]:
# env.add_robot()

In [ ]:
env.add_object(4, scale = 0.1)

In [ ]:
from omni.isaac.core.prims.xform_prim import XFormPrim
mobility_obj = XFormPrim("/World/Game/mobility")
mobility_obj_name = mobility_obj.name

In [ ]:
world.scene.add(mobility_obj)

In [ ]:
world.reset()
world.render()

In [ ]:
# stage = world.scene.stage
# list(stage.TraverseAll())

In [ ]:
env.get_image()

# Task

In [ ]:
from task.checker import TaskChecker
from task.instructor import SceneInstructor

In [ ]:
scene_instr = SceneInstructor()
scene_instr.analysis()
# scene_instr.build_handle_desc_ui()

In [ ]:
scene_instr.is_obj_valid

In [ ]:
scene_instr.valid_handle_list

In [ ]:
handle_path_str = list(scene_instr.valid_handle_list.keys())[2]

handle_joint_type = scene_instr.handle_knowledge[handle_path_str]["joint_type"]
handle_joint = scene_instr.handle_knowledge[handle_path_str]["joint_path_str"].split("/")[-1]


rel_direction = scene_instr.handle_knowledge[handle_path_str]["relative_to_game_center"]

In [ ]:
handle_path_str, handle_joint_type, handle_joint, rel_direction

In [ ]:
task_checker = TaskChecker("mobility", handle_joint, handle_joint_type, IS_RUNTIME=True)

# Solve

In [ ]:
# init
world.reset()
controller.start()
# world.scene.add(controller.robots)

In [ ]:
# get grasp location
goal_pos, goal_rot = controller.calculate_grasp_location(keyword = handle_path_str, verticle = True)

In [ ]:
goal_pos, goal_rot

In [ ]:
# move above
goal_pos[...,0] -= 0.15
controller.move_hand_to_fast(goal_pos, goal_rot, world, max_step=300)

In [ ]:
# move to
goal_pos[...,0] += 0.15
controller.move_hand_to_slow(goal_pos, goal_rot, world, step = 90)

In [ ]:
# close gripper
for i in range(60):
    world.step(render=True)

    u = controller.move_to_target(goal_pos, goal_rot)
    u[:,[-2, -1]] = 0.5 - (0.5 - -1) / 60 * i
    controller.robots.set_joint_position_targets(u)

In [ ]:
if handle_joint_type == "PhysicsRevoluteJoint": # door
    if rel_direction == "right":
        # 1. Open closewise

        # rotate and pull
        for i in range(0, 60):
            world.step(render=True)

            target_pos, target_rot = controller.calculate_pull_location(goal_pos, goal_rot, np.pi / 180 * i, 0.3, clock_wise=True)
            u = controller.move_to_target(target_pos, target_rot)
            u[:,[-2, -1]] = -1
            controller.robots.set_joint_position_targets(u)

    else:
        # 2. Open counter-closewise

        # rotate and pull
        for i in range(0, 60):
            world.step(render=True)

            target_pos, target_rot = controller.calculate_pull_location(goal_pos, goal_rot, np.pi / 180 * i, 0.3, 
                                                                        clock_wise=False)
            u = controller.move_to_target(target_pos, target_rot)
            u[:,[-2, -1]] = -1
            controller.robots.set_joint_position_targets(u)
            
else: # drawer
    # pull only
    for i in range(60):
        world.step(render=True)

        goal_pos[...,0] -= 2e-4 * i
        u = controller.move_to_target(goal_pos, goal_rot)
        u[:,[-2, -1]] = -1
        controller.robots.set_joint_position_targets(u)

In [ ]:
# check task
task_checker.joint_checker.compute_percentage()

In [ ]:
world.render()

In [ ]:
env.get_image()

# Close

In [ ]:
world.reset()

In [ ]:
world.scene.remove_object(mobility_obj_name)

# omni.kit.commands.execute("DeletePrims", paths=["/World/Game/mobility"])

In [ ]:
world.render()

In [ ]:
world.reset()

In [ ]:
env.get_image()